<a href="https://colab.research.google.com/github/jhonnycodedev/data-science/blob/jhonnycodedev/Lista%20NPS%20Positivo_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd


In [4]:
from google.colab import files
uploaded = files.upload()


Saving Lista NPS Positivo_V4.xlsx to Lista NPS Positivo_V4.xlsx


In [6]:
import io

# Substitua 'nome_do_arquivo.xlsx' pelo nome do seu arquivo
df = pd.read_excel(io.BytesIO(uploaded['Lista NPS Positivo_V4.xlsx']), engine='openpyxl')


In [7]:
# Verificando as primeiras linhas
print(df.head())

# Resumo estatístico
print(df.describe())

# Informações sobre o DataFrame
print(df.info())


                      db-id control_id                 invite_id Tipo Produto  \
0  620a60fc8e6c63645ad9ca1f     HNFX8D  620a608ab0d07664995bd771       Tipo 1   
1  620a6118a152566455632067     DN40ZZ  620a6088b0d07664995b80e0       Tipo 1   
2  620a61668e6c63645ad9cb7d     JQMNS6  620a6089b0d07664995bb9b8       Tipo 1   
3  620a616f6c698d64297cf7cd     XWHPG9  620a608bb0d07664995c01d3       Tipo 1   
4  620a62c66c698d64297d0446     SJV5N6  620a6089b0d07664995bbd1c       Tipo 1   

  Grupo de Produto Familia do Produto Modelo do Produto Chassi do Produto  \
0          Grupo 1          Familia 1        Modelo 116          Chassi 1   
1          Grupo 1          Familia 2        Modelo 120          Chassi 2   
2          Grupo 1          Familia 3        Modelo 118          Chassi 3   
3          Grupo 1          Familia 4         Modelo 80          Chassi 1   
4          Grupo 2          Familia 5        Modelo 110          Chassi 5   

  Marca do Produto           nome  ...  \
0       